In [1]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import json
from census import Census

In [2]:
# Read the Net_generation_for_all_sectors_monthly CSV file 
data = pd.read_csv('Net_generation_for_all_sectors_.csv', sep=',', skiprows=4)

# Read the renewable_energy_consumption_source CSV file
renewable_energy_consumption = pd.read_csv('renewable_energy_consumption_source.csv')

transposed_data = data.transpose()

# Rename columns with values from the description row
transposed_data.columns = transposed_data.iloc[0]

# Drop the non-numeric rows of data
transposed_data = transposed_data.drop(['description','units', 'source key'])

# Delete 'Net generation for all sectors' and 'United States' columns
del transposed_data['Net generation for all sectors']
del transposed_data['United States']

# Rename 'decription' column to 'date'
transposed_data.index.name = 'date'

# Add an index column
transposed_data.reset_index(inplace = True) 

# Display cleaned_data DataFrame the modified data
transposed_data.head()

description,date,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,...,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
0,Jan 2010,360957,173320,3187,1161,74173,909,72569,22383,NaN,...,10,1312,4629,3126,1503,-565,1014,--,--,10
1,Feb 2010,319735,153044,1251,1122,66198,825,65245,20590,NaN,...,33,1159,4277,2895,1382,-351,909,--,--,33
2,Mar 2010,312168,144406,1272,1198,63431,1010,64635,20886,NaN,...,76,1307,4682,3090,1592,-325,1002,--,--,76
3,Apr 2010,287800,126952,1220,1067,64644,943,57611,19097,NaN,...,112,1240,4490,2932,1558,-335,996,--,--,112
4,May 2010,327936,143272,1851,1143,73665,1017,66658,25079,NaN,...,153,1311,4470,2893,1577,-441,1060,--,--,153


In [3]:
data.head()

,description,units,source key,Jan 2010,Feb 2010,Mar 2010,Apr 2010,May 2010,Jun 2010,Jul 2010,...,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020
0,Net generation for all sectors,thousand megawatthours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,NaN,ELEC.GEN..M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States : all fuels (utility-scale),thousand megawatthours,ELEC.GEN.ALL-US-99.M,360957,319735,312168,287800,327936,375759,409725,...,309870.0,279846.0,304837.0,351967.0,409871.0,398536.0,333493.0,313703.0,301403.0,344523.0
3,United States : coal,thousand megawatthours,ELEC.GEN.COW-US-99.M,173320,153044,144406,126952,143272,165491,179600,...,50731.0,40675.0,46527.0,65283.0,89709.0,91145.0,68407.0,59805.0,61182.0,78588.0
4,United States : petroleum liquids,thousand megawatthours,ELEC.GEN.PEL-US-99.M,3187,1251,1272,1220,1851,2656,2970,...,691.0,626.0,691.0,818.0,914.0,887.0,755.0,876.0,800.0,940.0


In [4]:
energy_data_df = transposed_data.copy()

In [5]:
energy_data_df = energy_data_df.fillna(0)
energy_data_df = energy_data_df.replace('--',0)
energy_data_df.head()

description,date,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,...,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
0,Jan 2010,360957,173320,3187,1161,74173,909,72569,22383,0,...,10,1312,4629,3126,1503,-565,1014,0.0,0.0,10
1,Feb 2010,319735,153044,1251,1122,66198,825,65245,20590,0,...,33,1159,4277,2895,1382,-351,909,0.0,0.0,33
2,Mar 2010,312168,144406,1272,1198,63431,1010,64635,20886,0,...,76,1307,4682,3090,1592,-325,1002,0.0,0.0,76
3,Apr 2010,287800,126952,1220,1067,64644,943,57611,19097,0,...,112,1240,4490,2932,1558,-335,996,0.0,0.0,112
4,May 2010,327936,143272,1851,1143,73665,1017,66658,25079,0,...,153,1311,4470,2893,1577,-441,1060,0.0,0.0,153


In [6]:
energy_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 21 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   date                                            132 non-null    object 
 1   United States : all fuels (utility-scale)       132 non-null    object 
 2   United States : coal                            132 non-null    object 
 3   United States : petroleum liquids               132 non-null    object 
 4   United States : petroleum coke                  132 non-null    object 
 5   United States : natural gas                     132 non-null    object 
 6   United States : other gases                     132 non-null    object 
 7   United States : nuclear                         132 non-null    object 
 8   United States : conventional hydroelectric      132 non-null    object 
 9   United States : other renewables           

In [7]:
energy_data_df.columns

Index(['date', 'United States : all fuels (utility-scale)',
       'United States : coal', 'United States : petroleum liquids',
       'United States : petroleum coke', 'United States : natural gas',
       'United States : other gases', 'United States : nuclear',
       'United States : conventional hydroelectric',
       'United States : other renewables', 'United States : wind',
       'United States : all utility-scale solar', 'United States : geothermal',
       'United States : biomass',
       'United States : wood and wood-derived fuels',
       'United States : other biomass',
       'United States : hydro-electric pumped storage',
       'United States : other', 'United States : all solar',
       'United States : small-scale solar photovoltaic',
       'United States : all utility-scale solar'],
      dtype='object', name='description')

In [8]:
# energy_data_df.iloc[1:] = energy_data_df.iloc[1:].astype(int)
# energy_data_df.info()

In [9]:
# energy_data_df[['United States : all fuels (utility-scale)', 'United States : coal',
#        'United States : petroleum liquids', 'United States : petroleum coke',
#        'United States : natural gas', 'United States : other gases',
#        'United States : nuclear', 'United States : conventional hydroelectric',
#        'United States : other renewables', 'United States : wind',
#        'United States : all utility-scale solar', 'United States : geothermal',
#        'United States : biomass']] = energy_data_df[['United States : all fuels (utility-scale)', 'United States : coal',
#        'United States : petroleum liquids', 'United States : petroleum coke',
#        'United States : natural gas', 'United States : other gases',
#        'United States : nuclear', 'United States : conventional hydroelectric',
#        'United States : other renewables', 'United States : wind',
#        'United States : all utility-scale solar', 'United States : geothermal',
#        'United States : biomass']].apply(pd.to_numeric)

In [10]:
# energy_data_df[['United States : all fuels (utility-scale)', 
#                 'United States : coal',
#                'United States : other', 'United States : all solar']] = energy_data_df[['United States : all fuels (utility-scale)', 
#                                                            'United States : coal',
#                                                             'United States : other', 'United States : all solar']].astype(int)


In [11]:
energy_data_df['date'] = pd.to_datetime(energy_data_df['date'])
energy_data_df['United States : all fuels (utility-scale)'] = energy_data_df['United States : all fuels (utility-scale)'].astype(int)
energy_data_df['United States : coal'] = energy_data_df['United States : coal'].astype(int)
energy_data_df['United States : petroleum liquids'] = energy_data_df['United States : petroleum liquids'].astype(int)
energy_data_df['United States : petroleum coke'] = energy_data_df['United States : petroleum coke'].astype(int)
energy_data_df['United States : natural gas'] = energy_data_df['United States : natural gas'].astype(int)
energy_data_df['United States : other gases'] = energy_data_df['United States : other gases'].astype(int)
energy_data_df['United States : nuclear'] = energy_data_df['United States : nuclear'].astype(int)
energy_data_df['United States : conventional hydroelectric'] = energy_data_df['United States : conventional hydroelectric'].astype(int)
energy_data_df['United States : wind'] = energy_data_df['United States : wind'].astype(int)
energy_data_df['United States : all utility-scale solar'] = energy_data_df['United States : all utility-scale solar'].astype(int)
energy_data_df['United States : geothermal'] = energy_data_df['United States : geothermal'].astype(int)
energy_data_df['United States : biomass'] = energy_data_df['United States : biomass'].astype(int)
energy_data_df['United States : wood and wood-derived fuels'] = energy_data_df['United States : wood and wood-derived fuels'].astype(int)
energy_data_df['United States : other biomass'] = energy_data_df['United States : other biomass'].astype(int)
energy_data_df['United States : hydro-electric pumped storage'] = energy_data_df['United States : hydro-electric pumped storage'].astype(int)
energy_data_df['United States : other'] = energy_data_df['United States : other'].astype(int)
energy_data_df['United States : all solar'] = energy_data_df['United States : all solar'].astype(int)
energy_data_df['United States : small-scale solar photovoltaic'] = energy_data_df['United States : small-scale solar photovoltaic'].astype(int)
energy_data_df['United States : all utility-scale solar'] = energy_data_df['United States : all utility-scale solar'].astype(int)

energy_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 21 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   date                                            132 non-null    datetime64[ns]
 1   United States : all fuels (utility-scale)       132 non-null    int64         
 2   United States : coal                            132 non-null    int64         
 3   United States : petroleum liquids               132 non-null    int64         
 4   United States : petroleum coke                  132 non-null    int64         
 5   United States : natural gas                     132 non-null    int64         
 6   United States : other gases                     132 non-null    int64         
 7   United States : nuclear                         132 non-null    int64         
 8   United States : conventional hydroelectric      13

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_11972/3354050356.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  energy_data_df['date'] = pd.to_datetime(energy_data_df['date'])


In [12]:
energy_data_df.iloc[0][0]

Timestamp('2010-01-01 00:00:00')

In [21]:
year_2010 = energy_data_df[(energy_data_df['date'] >= '2010-01-01') & (energy_data_df['date'] < '2011-01-01')]
year_2011 = energy_data_df[(energy_data_df['date'] >= '2011-01-01') & (energy_data_df['date'] < '2012-01-01')]
year_2012 = energy_data_df[(energy_data_df['date'] >= '2012-01-01') & (energy_data_df['date'] < '2013-01-01')]
year_2013 = energy_data_df[(energy_data_df['date'] >= '2013-01-01') & (energy_data_df['date'] < '2014-01-01')]
year_2014 = energy_data_df[(energy_data_df['date'] >= '2014-01-01') & (energy_data_df['date'] < '2015-01-01')]
year_2015 = energy_data_df[(energy_data_df['date'] >= '2015-01-01') & (energy_data_df['date'] < '2016-01-01')]
year_2016 = energy_data_df[(energy_data_df['date'] >= '2016-01-01') & (energy_data_df['date'] < '2017-01-01')]
year_2017 = energy_data_df[(energy_data_df['date'] >= '2017-01-01') & (energy_data_df['date'] < '2018-01-01')]
year_2018 = energy_data_df[(energy_data_df['date'] >= '2018-01-01') & (energy_data_df['date'] < '2019-01-01')]
year_2019 = energy_data_df[(energy_data_df['date'] >= '2019-01-01') & (energy_data_df['date'] < '2020-01-01')]
year_2020 = energy_data_df[(energy_data_df['date'] >= '2020-01-01') & (energy_data_df['date'] < '2021-01-01')]


# Select only numerical columns (excluding the 'date' column)
numerical_columns = energy_data_df.select_dtypes(include=['int64', 'float64'])

# Calculate the sum of each numerical column
sum_of_columns_2010 = year_2010[numerical_columns.columns].sum()
sum_of_columns_2011 = year_2011[numerical_columns.columns].sum()
sum_of_columns_2012 = year_2012[numerical_columns.columns].sum()
sum_of_columns_2013 = year_2013[numerical_columns.columns].sum()
sum_of_columns_2014 = year_2014[numerical_columns.columns].sum()
sum_of_columns_2015 = year_2015[numerical_columns.columns].sum()
sum_of_columns_2016 = year_2016[numerical_columns.columns].sum()
sum_of_columns_2017 = year_2017[numerical_columns.columns].sum()
sum_of_columns_2018 = year_2018[numerical_columns.columns].sum()
sum_of_columns_2019 = year_2019[numerical_columns.columns].sum()
sum_of_columns_2020 = year_2020[numerical_columns.columns].sum()


# Create a dictionary with the sum of columns for each year
data = {
    '2010_Totals': sum_of_columns_2010,
    '2011_Totals': sum_of_columns_2011,
    '2012_Totals': sum_of_columns_2012,
    '2013_Totals': sum_of_columns_2013,
    '2014_Totals': sum_of_columns_2014,
    '2015_Totals': sum_of_columns_2015,
    '2016_Totals': sum_of_columns_2016,
    '2017_Totals': sum_of_columns_2017,
    '2018_Totals': sum_of_columns_2018,
    '2019_Totals': sum_of_columns_2019,
    '2020_Totals': sum_of_columns_2020,
}

# Create a DataFrame from the dictionary
energy_data_totals_df = pd.DataFrame(data)

# Display the DataFrame
energy_data_totals_df.head()

,2010_Totals,2011_Totals,2012_Totals,2013_Totals,2014_Totals,2015_Totals,2016_Totals,2017_Totals,2018_Totals,2019_Totals,2020_Totals
description,,,,,,,,,,,
United States : all fuels (utility-scale),4125059,4100141,4047765,4065965,4093563,4078714,4077573,4035443,4180987,4130575,4009766
United States : coal,1847289,1733431,1514043,1581116,1581710,1352400,1239149,1205835,1149487,964958,773393
United States : petroleum liquids,23337,16086,13400,13820,18275,17373,13008,12413,16244,11523,9662
United States : petroleum coke,13725,14096,9787,13344,11955,10877,11198,8975,8982,6819,7679
United States : natural gas,987696,1013689,1225895,1124834,1126635,1334669,1379271,1297701,1471843,1588532,1626790


In [22]:
energy_data_totals_df.iloc[0]

2010_Totals    4125059
2011_Totals    4100141
2012_Totals    4047765
2013_Totals    4065965
2014_Totals    4093563
2015_Totals    4078714
2016_Totals    4077573
2017_Totals    4035443
2018_Totals    4180987
2019_Totals    4130575
2020_Totals    4009766
Name: United States : all fuels (utility-scale), dtype: int64

In [15]:
renewable_energy_consumption.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,WDPRBUS,194913,1549.262,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,195013,1562.307,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,195113,1534.669,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,195213,1474.369,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,195313,1418.601,1,Wood Energy Production,Trillion Btu


In [16]:
# Convert 'YYYYMM' column to string
renewable_energy_consumption['YYYYMM'] = renewable_energy_consumption['YYYYMM'].astype(str)

# Filter out rows where 'YYYYMM' ends with '13'
renewable_energy_consumption_df = renewable_energy_consumption[~renewable_energy_consumption['YYYYMM'].str.endswith('13')]

# Change 'YYYYMM' column name to 'Date'
renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)

# Convert the 'Date' column to a datetime format
renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
# df['date_column'] = pd.to_datetime(df['date_column'].astype(str), format='%Y%m%d')

# Convert the 'Value' column to a numeric format
renewable_energy_consumption_df['Value'] = renewable_energy_consumption_df['Value'].replace('Not Available', pd.NA)
renewable_energy_consumption_df['Value'] = pd.to_numeric(renewable_energy_consumption_df['Value'], errors='coerce')

renewable_energy_consumption_df.head()

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_11972/4111504033.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_11972/4111504033.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_11972/4111504033.py:15: SettingWithCopyWarning:

,MSN,Date,Value,Column_Order,Description,Unit
24,WDPRBUS,1973-01-01,129.630,1,Wood Energy Production,Trillion Btu
25,WDPRBUS,1973-02-01,117.194,1,Wood Energy Production,Trillion Btu
26,WDPRBUS,1973-03-01,129.763,1,Wood Energy Production,Trillion Btu
27,WDPRBUS,1973-04-01,125.462,1,Wood Energy Production,Trillion Btu
28,WDPRBUS,1973-05-01,129.624,1,Wood Energy Production,Trillion Btu


In [17]:
renewable_energy_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7956 entries, 24 to 8929
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   MSN           7956 non-null   object        
 1   Date          7956 non-null   datetime64[ns]
 2   Value         7512 non-null   float64       
 3   Column_Order  7956 non-null   int64         
 4   Description   7956 non-null   object        
 5   Unit          7956 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 435.1+ KB


In [18]:
# Filter data for all dates from year >= 2018
filtered_renewables_df = renewable_energy_consumption_df[(renewable_energy_consumption_df['Date'].dt.year >= 2010) & 
                                              (renewable_energy_consumption_df['Date'].dt.year <= 2020)]
# Reset index
clean_filtered_renewables_df = filtered_renewables_df.reset_index(drop=True)

clean_filtered_renewables_df.head()

,MSN,Date,Value,Column_Order,Description,Unit
0,WDPRBUS,2010-01-01,187.178,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,2010-02-01,171.797,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,2010-03-01,187.220,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,2010-04-01,179.128,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,2010-05-01,181.788,1,Wood Energy Production,Trillion Btu


In [20]:
# Extract year from 'Date' column
clean_filtered_renewables_df['Year'] = clean_filtered_renewables_df['Date'].dt.year

# Group by 'Description' and 'Year', and sum the 'Value' column
grouped_renewables_df = clean_filtered_renewables_df.groupby(['MSN', 'Year','Description', 'Unit'])['Value'].sum().reset_index()

# Sort the DataFrame by 'Year' and 'Description'
grouped_renewables_df = grouped_renewables_df.sort_values(by=['Year', 'Description'])
grouped_renewables_df.reset_index(drop=True, inplace=True)
grouped_renewables_df.head(15)

,MSN,Year,Description,Unit,Value
0,BFTCBUS,2010,Biofuels Consumption,Trillion Btu,1820.984
1,BFPRBUS,2010,Biofuels Production,Trillion Btu,1867.880
2,GETCBUS,2010,Geothermal Energy Consumption,Trillion Btu,111.427
3,HVTCBUS,2010,Hydroelectric Power Consumption,Trillion Btu,887.812
4,SOTCBUS,2010,Solar Energy Consumption,Trillion Btu,67.853
5,BMTCBUS,2010,Total Biomass Energy Consumption,Trillion Btu,4505.636
6,BMPRBUS,2010,Total Biomass Energy Production,Trillion Btu,4552.531
7,RETCBUS,2010,Total Renewable Energy Consumption,Trillion Btu,5895.684
8,REPRBUS,2010,Total Renewable Energy Production,Trillion Btu,5942.579
9,WSTCBUS,2010,Waste Energy Consumption,Trillion Btu,467.911


In [24]:
# Export the 'energy_data_totals_df' and the 'grouped_renewables_df' as CSV files
energy_data_totals_df.to_csv('net_energy_generation_totals.csv', sep=',', index=False, encoding='utf-8')
grouped_renewables_df.to_csv('renewables_energy_annual_consumption.csv', sep=',', index=False, encoding='utf-8')